# Import

In [262]:
import transformers
import torch
import pandas as pd
from tqdm.auto import tqdm

from torch.utils.data import DataLoader

from transformers import AutoTokenizer, AutoModelForMaskedLM
from datasets import load_dataset
from transformers import DataCollatorForLanguageModeling

In [357]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [358]:
tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
model = AutoModelForMaskedLM.from_pretrained("beomi/kcbert-base").to(device)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [271]:
def tokenization(example):
    return tokenizer(example["text"])

train_dataset = load_dataset("csv", data_files="dataset/train.csv")['train']
train_dataset = train_dataset.map(tokenization, batched=True, remove_columns=["id", "text", "label"])
train_dataset

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2000
})

In [339]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=False, collate_fn=data_collator)
train_loader

In [359]:
def augment_epoch():
    augmented_texts = []

    for batch in tqdm(train_loader):
        batch = batch.to(device)
        token_logits = model(**batch).logits

        # Loop over each example in batch
        for i in range(batch.input_ids.shape[0]):
            mask_token_index = torch.where(batch.input_ids[i] == tokenizer.mask_token_id)[0]
            mask_token_logits = token_logits[i, mask_token_index, :]
            replaced_tokens = mask_token_logits.argmax(dim=1).tolist()

            # Replace mask tokens by replaced tokens
            augmented_input = batch.input_ids[i].clone()
            for k in range(len(mask_token_index)):
                augmented_input[mask_token_index[k]] = replaced_tokens[k]

            augmented_text = tokenizer.decode(augmented_input, skip_special_tokens=True)
            augmented_texts.append(augmented_text)

    return augmented_texts

In [354]:
train_df = pd.read_csv("dataset/train.csv")

In [370]:
# augmented_data = {
#     "id": train_df["id"].tolist(), 
#     "text": train_df["text"].tolist(),
#     "label": train_df["label"].tolist(),
#     }
augmented_data = {"id": [], "text": [], "label": []}
num_augs = 4

# To avoid duplicates, augment 3 times more than num_augs
for i in tqdm(range(num_augs * 2)):
    augmented_texts = augment_epoch()
    for j, row in train_df.iterrows():
        augmented_data["id"].append(f"{row['id']}")
        augmented_data["text"].append(augmented_texts[j])
        augmented_data["label"].append(row["label"])

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

In [361]:
augment_df = pd.DataFrame(augmented_data)
augment_df['id'] = augment_df['id'].astype(int)
augment_df.sort_values(by=["id"], inplace=True)
augment_df

,id,text,label
0,218,가성비 굿이에요 저는 프레임만 구입했는데 2주만에 왔고요 디자인 깔끔하고요 인터넷 ...,1
4000,218,가성비 굿입니다 저는 프레임만 구입했는데 2주만에 왔고요 디자인 깔끔하고요 인터넷 ...,1
22000,218,가성비 굿 전략 저는 프레임만 구입한지 오랫만에 왔고요 디자인 깔끔하고요 인터넷 다...,1
6000,218,가성비 굿이에요 저는 프레임만 구입했는데 2주만에 왔고요 디자인 깔끔했고요 인터넷 ...,1
20000,218,가성비 굿이에요 저는 프레임폰 구입했는데 2주만에 왔고요 디자인 깔끔하고요 인터넷 ...,1
...,...,...,...
3999,199719,진쨔 예쁩니다.. 두말 하면 잔소리에요 카고팬츠 엄청 사고싶어서 많이 봤는대 이거 ...,1
21999,199719,진쨔 예쁩니다.. 일밍아웃 하면 잔소리에요 카고팬츠 엄청 사고 그런거 많이 봤는대 ...,1
1999,199719,진쨔 예쁩니다.. 두말 하면 잔소리에요 카고팬츠 엄청 사고싶어서 많이 봤는대 이거 ...,1
7999,199719,진쨔 예쁩니다.. 두말하면 잔소리 자칭 카피팬츠 엄청 사고싶어서 많이 봤는대 그 만...,1


In [362]:
augment_df.drop_duplicates(subset=["text"], inplace=True)
augment_df

,id,text,label
0,218,가성비 굿이에요 저는 프레임만 구입했는데 2주만에 왔고요 디자인 깔끔하고요 인터넷 ...,1
4000,218,가성비 굿입니다 저는 프레임만 구입했는데 2주만에 왔고요 디자인 깔끔하고요 인터넷 ...,1
22000,218,가성비 굿 전략 저는 프레임만 구입한지 오랫만에 왔고요 디자인 깔끔하고요 인터넷 다...,1
6000,218,가성비 굿이에요 저는 프레임만 구입했는데 2주만에 왔고요 디자인 깔끔했고요 인터넷 ...,1
20000,218,가성비 굿이에요 저는 프레임폰 구입했는데 2주만에 왔고요 디자인 깔끔하고요 인터넷 ...,1
...,...,...,...
11999,199719,진교수님 예쁩니다.. 두말 하면 잔소리에요 카고팬츠 엄청 사고싶어서 많이 샀는대 이...,1
19999,199719,진쨔 예쁩니다.. 두말 하면 잔소리에요 카고팬츠 엄청보고싶어서 몇번 봤는대 이거 만...,1
21999,199719,진쨔 예쁩니다.. 일밍아웃 하면 잔소리에요 카고팬츠 엄청 사고 그런거 많이 봤는대 ...,1
7999,199719,진쨔 예쁩니다.. 두말하면 잔소리 자칭 카피팬츠 엄청 사고싶어서 많이 봤는대 그 만...,1


In [ ]:
train_df.sample

In [368]:
num_samples = len(train_df) * num_augs
labels = augment_df["label"].unique().tolist()
num_labels = len(labels)

aug_sampled_df = pd.DataFrame(columns=["id", "text", "label"])  
for label in labels:
    label_sampled_df = augment_df[augment_df["label"] == label].sample(n=num_samples // num_labels)
    aug_sampled_df = pd.concat([aug_sampled_df, label_sampled_df])

aug_sampled_df.sort_values(by=["id"], inplace=True)
aug_sampled_df

['장갑도 짱짱하고 패드도 좋아요. 장갑낀 상태에서 스마트폰 터치도 됩니다.',
 '장갑이 짱짱하고 패드도 좋아요. 장갑낀 상태에서 스마트폰 터치도 됩니다.',
 '장갑 뭉 짱짱하고 패딩도 됩니다.nd낀 상태에서 스마트폰 터 저도 됩니다.',
 '장갑이 짱짱하고 패드도 좋아요. 장갑낀 상태에서는장치도 됩니다.',
 '장갑이 짱짱하고 패드도 좋아요. 장갑낀 사람은 스마트폰 터 보내서 됩니다.',
 '오짱욘짱하고 패드도 좋아요. 장갑낀 상태에서 스마트폰 터치도 됩니다.',
 '장갑이 짱짱하고 패드도 좋아요. 장갑이 생각나는 스마트폰 터치도 됩니다.',
 '장갑끼고 짱짱하고 패드도 좋아요. 장갑낀 사람은 스마트폰 터치도 됩니다.',
 '장갑이 짱짱하고 패드도 좋아요. 장갑 찬 상태에서는 터치도 좋아요.']

In [ ]:
aug_df = pd.concat([train_df, aug_sampled_df])
aug_df.sort_values(by=["id"], inplace=True, kind="stable")
aug_df

In [363]:
augment_df['id'].value_counts()

id
218       13
83921     13
86198     13
85825     13
85731     13
          ..
68699      4
123107     4
44342      3
104098     2
187238     1
Name: count, Length: 2000, dtype: int64

In [317]:
augment_df.to_csv("dataset/train_aug_mlm.csv", index=False)